In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pwd

/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/BEAM Parser


In [16]:
import os
import sys
# cwd = os.getcwd()
cm_folder = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/"
models_folder = os.path.join(cm_folder, "BEAM Parser/models/")
src_path = os.path.join(cm_folder, "src")
sys.path.append(src_path)

In [27]:
from typing import Any

import dill
from sklearn.linear_model import LogisticRegression
import numpy as np

from CrossValidation import cross_validation
from MIRA import CostSensitiveMIRA
from Settings import Settings
from crel_helper import get_cr_tags
from crel_processing import essay_to_crels_cv
from evaluation import evaluate_model_essay_level, get_micro_metrics, metrics_to_df
from feature_extraction import get_features_from_probabilities
from feature_normalization import min_max_normalize_feats
from function_helpers import get_function_names
from results_procesor import ResultsProcessor
from train_parser import essay_to_crels, create_extractor_functions
from cost_functions import micro_f1_cost_plusepsilon
from train_reranker import train_model_parallel, train_model, train_cost_sensitive_instance
from window_based_tagger_config import get_config
from feature_extraction import get_features_essay_level

In [6]:
# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings
settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"

coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

config = get_config(training_folder)

train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
902 226


In [7]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)

set_cr_tags = set(cr_tags)
list(set_cr_tags)[0:10]

['Causer:4->Result:5b',
 'Causer:13->Result:12',
 'Causer:11->Result:4',
 'Causer:2->Result:7',
 'Causer:5b->Result:14',
 'Causer:6->Result:7',
 'Causer:4->Result:5',
 'Causer:5->Result:14',
 'Causer:3->Result:2',
 'Causer:7->Result:5']

In [8]:
from searn_essay_parser_breadth_first import SearnModelEssayParserBreadthFirst

In [9]:
test_folds = [(pred_tagged_essays_train, pred_tagged_essays_test)]  # type: List[Tuple[Any,Any]]
cv_folds = cross_validation(pred_tagged_essays_train, CV_FOLDS)  # type: List[Tuple[Any,Any]]

In [10]:
len(pred_tagged_essays_train)

902

### Get the Expected Crels Per Essay

In [29]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
name2essay = {}
for essay in all_essays:
    name2essay[essay.name] = essay

name2crels = essay_to_crels(all_essays, set_cr_tags)
assert len(name2crels) == len(all_essays)

## Train Re-Ranker

In [18]:
BEAM_SIZE = 10

In [21]:
import dill

rr_fname = "xs_rerank_" + str(BEAM_SIZE) + ".dill"
with open(os.path.join(models_folder, rr_fname), "rb") as f:
    xs_rr = dill.load(f)
len(xs_rr)

902

In [22]:
assert len(xs_rr) == len(pred_tagged_essays_train)

In [24]:
len(xs_rr), len(pred_tagged_essays_train)

(902, 902)

In [30]:
%%time
xs = get_features_essay_level(xs_rr, name2crels, min_feat_freq=1)

cv_folds_rerank = cross_validation(xs, 5)
cv_folds_mm = [min_max_normalize_feats(train, test) for (train, test) in cv_folds_rerank]

CPU times: user 8.25 s, sys: 111 ms, total: 8.36 s
Wall time: 8.39 s


In [36]:
best_C, best_max_upd = 0.01, 1

In [38]:
%%time
f1 = train_model_parallel(cv_folds=cv_folds_mm, name2essay=name2essay, C=best_C, pa_type=1, loss_type="ml", \
                          max_update_items=best_max_upd, set_cr_tags=set_cr_tags)
print(f1)  # 0.7421167703055035

0.7372937177905368
CPU times: user 9.4 s, sys: 714 ms, total: 10.1 s
Wall time: 1min 9s


## Train on Test Data

In [116]:
xs_test_rerank = essay_to_crels_cv(test_folds, final_test_model, top_n=best_top_n, search_mode_max_prob=False)
xs_test = get_features_from_probabilities(xs_test_rerank, name2crels, best_max_parses, min_feat_freq=1,
                                          min_prob=best_min_prob)

In [117]:
# training data comes from the test fold predictions from CV on the full training dataset
xs_train = []
for train, test in cv_folds_rerank:
    xs_train.extend(test)

In [118]:
# Normalize both using training data
xs_train_mm, xs_test_mm = min_max_normalize_feats(xs_train,xs_test)

In [119]:
num_train = int(0.8 * len(xs_train_mm))
tmp_train_copy = list(xs_train_mm)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

In [120]:
%%time
C = best_C
pa_type = 1
loss_type= "ml"
max_update_items = best_max_upd

mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=5, train_instance_fn = train_cost_sensitive_instance, verbose=True)

Epoch: 0 Train Accuracy: 0.7404 Test Accuracy: 0.7276
Epoch: 1 Train Accuracy: 0.7447 Test Accuracy: 0.7262
Epoch: 2 Train Accuracy: 0.7454 Test Accuracy: 0.7262
Epoch: 3 Train Accuracy: 0.7466 Test Accuracy: 0.7268
Epoch: 4 Train Accuracy: 0.7463 Test Accuracy: 0.7274
Epoch: 5 Train Accuracy: 0.7459 Test Accuracy: 0.7274
Best Test Acc: 0.7276
CPU times: user 1min 9s, sys: 189 ms, total: 1min 9s
Wall time: 1min 9s


In [121]:
best_iterations

1

In [122]:
mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  xs_train=xs_train_mm, xs_test=xs_test_mm,
                                       name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=best_iterations, early_stop_iters=best_iterations, train_instance_fn = train_cost_sensitive_instance, verbose=True)

Epoch: 0 Train Accuracy: 0.7377 Test Accuracy: 0.6428
Best Test Acc: 0.6428


In [123]:
from filter_features import filter_feats

prefixes = [
    "Prob-",
    "CREL_Pair-",
    "Inv-",
    "num_crels",
    "Tally-",
    "CChain-",
    "CChainStats-",
    "Above-",
    "CREL_"
]
# xs_fltr_train, xs_fltr_test = filter_feats(xs_train_mm, xs_test_mm, prefixes)
assert len(prefixes) == len(set(prefixes)), "Duplicate prefixes found"

In [124]:
best_f1 = -1
current_best = []
remaining = list(prefixes)

while True:
    if len(remaining) == 0:
        break
    
    f1_by_prefix = dict()
    for prefix in remaining:
        new_prefixes = current_best + [prefix]
        
        cv_filtered = []
        for tr, test in cv_folds_mm:
            x_tr,x_test = filter_feats(tr, test, new_prefixes)
            cv_filtered.append((x_tr,x_test))
        
        f1_by_prefix[prefix] = train_model_parallel(cv_folds=cv_filtered, name2essay=name2essay, C=best_C, 
                                  pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
                                  set_cr_tags=set_cr_tags)
    
    best_prefix, new_best_f1 = sorted(f1_by_prefix.items(), key = lambda tpl: -tpl[1])[0]
    if new_best_f1 > best_f1:
        best_f1 = new_best_f1
        current_best.append(best_prefix)
        remaining.remove(best_prefix)
        print("{length} feats, new Best F1: {f1:.4f} Prefixes: {prefixes}".format(
            length=len(current_best), f1=best_f1, prefixes=str(current_best)))
    else:
        print("No further improvement, stopping")
        break

1 feats, new Best F1: 0.7327 Prefixes: ['Above-']
2 feats, new Best F1: 0.7337 Prefixes: ['Above-', 'CChainStats-']
3 feats, new Best F1: 0.7341 Prefixes: ['Above-', 'CChainStats-', 'num_crels']
4 feats, new Best F1: 0.7342 Prefixes: ['Above-', 'CChainStats-', 'num_crels', 'Tally-']
No further improvement, stopping


In [125]:
cv_filtered = []
for tr, test in cv_folds_mm:
    x_tr,x_test = filter_feats(tr, test, prefixes)
    cv_filtered.append((x_tr,x_test))

f1 = train_model_parallel(cv_folds=cv_filtered, name2essay=name2essay, C=best_C, 
                          pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
                          set_cr_tags=set_cr_tags)
f1

0.7307287791063584

## Apply to Test Data

In [126]:
current_best

['Above-', 'CChainStats-', 'num_crels', 'Tally-']

In [127]:
xs_train_mm_fltr, xs_test_mm_fltr = filter_feats(xs_train_mm, xs_test_mm, current_best) 

In [128]:
num_train = int(0.8 * len(xs_train_mm_fltr))
tmp_train_copy = list(xs_train_mm_fltr)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

In [129]:
%%time
C = best_C
pa_type = 1
loss_type= "ml"
max_update_items = best_max_upd

mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=3, train_instance_fn = train_cost_sensitive_instance, verbose=True)

Epoch: 0 Train Accuracy: 0.7234 Test Accuracy: 0.7185
Epoch: 1 Train Accuracy: 0.7301 Test Accuracy: 0.7224
Epoch: 2 Train Accuracy: 0.7355 Test Accuracy: 0.7257
Epoch: 3 Train Accuracy: 0.7370 Test Accuracy: 0.7290
Epoch: 4 Train Accuracy: 0.7385 Test Accuracy: 0.7279
Epoch: 5 Train Accuracy: 0.7388 Test Accuracy: 0.7290
Epoch: 6 Train Accuracy: 0.7387 Test Accuracy: 0.7290
Best Test Acc: 0.7290
CPU times: user 30.8 s, sys: 112 ms, total: 30.9 s
Wall time: 30.9 s


In [130]:
best_iterations 

4

In [131]:
mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm_fltr, xs_test=xs_test_mm_fltr,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=best_iterations, early_stop_iters=best_iterations, 
    train_instance_fn = train_cost_sensitive_instance, verbose=True)

Epoch: 0 Train Accuracy: 0.7192 Test Accuracy: 0.5120
Epoch: 1 Train Accuracy: 0.7281 Test Accuracy: 0.5622
Epoch: 2 Train Accuracy: 0.7322 Test Accuracy: 0.5995
Epoch: 3 Train Accuracy: 0.7325 Test Accuracy: 0.6299
Best Test Acc: 0.6299


In [132]:
sorted(best_mdl.weights.items(), key = lambda tpl: -tpl[1])[0:20]

[('num_crels=0', 0.9562086306995793),
 ('Above-%-0.9', 0.5597742798338828),
 ('Above-All-Above-0.3', 0.489293906141792),
 ('Above-All-Above-0.2', 0.469293906141792),
 ('Above-0.9', 0.3613245725798157),
 ('Above-%-0.95', 0.35014824203951467),
 ('Above-0.95', 0.3265583965373035),
 ('Above-%-0.8', 0.3127190332735536),
 ('Above-All-Above-0.5', 0.289073579080481),
 ('Above-All-Above-0.7', 0.2792795085997428),
 ('Above-0.8', 0.2604579132952751),
 ('Above-All-Above-0.9', 0.2476732861308604),
 ('Above-%-0.7', 0.23623862643125776),
 ('Above-0.7', 0.21291975385400053),
 ('Tally-Result:14', 0.20250000000000012),
 ('Tally-Causer:7', 0.1863206598212016),
 ('num_crels<=7', 0.1857981169585543),
 ('CChainStats-MaxChain_Len=5', 0.15794072834985382),
 ('Tally-Causer:13', 0.15250000000000008),
 ('num_crels=7', 0.1520935574085751)]

# Notes on Remaining Code Changes
- The Beam search approach outputs a list of Dict[str, List[float]], instead of just one Dict[str, List[float]]
- However, we don't need to sample from the crels, we will just use the already generated parses, after de-duping
- ParserInputs needs modifying so that it takes a list of crel2probs instead of on dict for all parses
- Need to figure out what the optimal parse is based on amount of overlap with the actual crels, minus the false positives

# TODO 
- include the cum prob from the parse action result as a feature? - or simply compute the geometric mean of the probs?
- To speed up MIRA, de-dupe the generated parses prior to feature extraction. Where there are dupes, take the one with the highest cum prob
